# 🚀 Model Serving with FastAPI

## Production ML Deployment

Turn your ML model into a production API in minutes.

---


In [ ]:
print('✅ FastAPI serving guide ready!')


## Why FastAPI for ML?

✅ **Fast**: Async/await support
✅ **Auto docs**: Swagger UI built-in
✅ **Type hints**: Pydantic validation
✅ **Production-ready**: Used by Uber, Netflix

## Basic ML API

```python
from fastapi import FastAPI
from pydantic import BaseModel
import joblib

app = FastAPI()
model = joblib.load('model.pkl')

class PredictionInput(BaseModel):
    features: list[float]

@app.post('/predict')
def predict(input: PredictionInput):
    prediction = model.predict([input.features])
    return {'prediction': prediction[0]}
```

## Running

```bash
uvicorn main:app --reload
```

API now at: http://localhost:8000
Docs at: http://localhost:8000/docs


## Production Best Practices

### 1. Request Validation

```python
class Features(BaseModel):
    age: int = Field(ge=0, le=120)
    income: float = Field(gt=0)
    
    @validator('age')
    def check_age(cls, v):
        if v < 18:
            raise ValueError('Must be 18+')
        return v
```

### 2. Error Handling

```python
@app.exception_handler(Exception)
async def global_exception_handler(request, exc):
    return JSONResponse(
        status_code=500,
        content={'error': str(exc)}
    )
```

### 3. Health Check

```python
@app.get('/health')
def health():
    return {'status': 'healthy', 'model_loaded': model is not None}
```

### 4. Batch Predictions

```python
@app.post('/predict/batch')
def predict_batch(inputs: list[PredictionInput]):
    features = [inp.features for inp in inputs]
    predictions = model.predict(features)
    return {'predictions': predictions.tolist()}
```


## Performance Optimization

### Async Endpoints

```python
@app.post('/predict')
async def predict(input: PredictionInput):
    # Use asyncio for I/O-bound ops
    result = await async_model_predict(input)
    return result
```

### Caching

```python
from functools import lru_cache

@lru_cache(maxsize=1000)
def cached_predict(features_tuple):
    return model.predict([list(features_tuple)])[0]
```

### Model Loading

```python
@app.on_event('startup')
async def load_model():
    global model
    model = joblib.load('model.pkl')
```
